In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import re as re
import os.path

import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit


# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.callbacks import History,LearningRateScheduler
# from tensorflow.keras.layers import Dropout
# print('TensorFlow %s, Keras %s, numpy %s, pandas %s'%(tf.__version__,keras.__version__, np.__version__,pd.__version__))
__DEBUG__=False

train = pd.read_csv('data_concatener.csv', sep=';')
test  = pd.read_csv('test_set_values.csv')
full_data = [train, test]
finalfile_index=test.id #Index des données de test pour le résultat final

# Useless column
train.drop('num_private', axis=1, inplace=True)
train.drop('date_recorded', axis=1, inplace=True)
train.drop('funder', axis=1, inplace=True)
train.drop('installer', axis=1, inplace=True)
train.drop('scheme_name', axis=1, inplace=True)
train.drop('scheme_management', axis=1, inplace=True)
train.drop('recorded_by', axis=1, inplace=True)
train.drop('wpt_name', axis=1, inplace=True)
train.drop('subvillage', axis=1, inplace=True)
test.drop('num_private', axis=1, inplace=True)
test.drop('date_recorded', axis=1, inplace=True)
test.drop('funder', axis=1, inplace=True)
test.drop('installer', axis=1, inplace=True)
test.drop('scheme_name', axis=1, inplace=True)
test.drop('scheme_management', axis=1, inplace=True)
test.drop('recorded_by', axis=1, inplace=True)
test.drop('subvillage', axis=1, inplace=True)
test.drop('wpt_name', axis=1, inplace=True)

# Double column
train.drop('quantity', axis=1, inplace=True)
train.drop('source', axis=1, inplace=True)
train.drop('waterpoint_type', axis=1, inplace=True)
train.drop('payment', axis=1, inplace=True)
train.drop('management', axis=1, inplace=True)
train.drop('extraction_type', axis=1, inplace=True)
train.drop('extraction_type_group', axis=1, inplace=True)
train.drop('water_quality', axis=1, inplace=True)
train.drop('source_type', axis=1, inplace=True)
train.drop('ward', axis=1, inplace=True)
train.drop('lga', axis=1, inplace=True)
train.drop('region', axis=1, inplace=True)
train.drop('basin', axis=1, inplace=True)
train.drop('longitude', axis=1, inplace=True)
train.drop('latitude', axis=1, inplace=True)
test.drop('quantity', axis=1, inplace=True)
test.drop('source', axis=1, inplace=True)
test.drop('waterpoint_type', axis=1, inplace=True)
test.drop('payment', axis=1, inplace=True)
test.drop('management', axis=1, inplace=True)
test.drop('extraction_type', axis=1, inplace=True)
test.drop('extraction_type_group', axis=1, inplace=True)
test.drop('water_quality', axis=1, inplace=True)
test.drop('source_type', axis=1, inplace=True)
test.drop('ward', axis=1, inplace=True)
test.drop('lga', axis=1, inplace=True)
test.drop('region', axis=1, inplace=True)
test.drop('basin', axis=1, inplace=True)
test.drop('longitude', axis=1, inplace=True)
test.drop('latitude', axis=1, inplace=True)

# Refactoring value empty
for dataset in full_data:
    dataset.loc[dataset.public_meeting.isnull(), 'public_meeting'] = 0
    dataset.loc[dataset.public_meeting == False, 'public_meeting'] = 0
    dataset.loc[dataset.public_meeting == True, 'public_meeting'] = 1
    # dataset.loc[dataset.latitude == 0.0, 'latitude'] = 35.225766
    # dataset.loc[dataset.longitude == '-2,00E-08', 'longitude'] = -6.145943
    dataset.loc[dataset.extraction_type_class == 'gravity', 'extraction_type_class'] = 0
    dataset.loc[dataset.extraction_type_class == 'submersible', 'extraction_type_class'] = 1
    dataset.loc[dataset.extraction_type_class == 'handpump', 'extraction_type_class'] = 2
    dataset.loc[dataset.extraction_type_class == 'other', 'extraction_type_class'] = 3
    dataset.loc[dataset.extraction_type_class == 'motorpump', 'extraction_type_class'] = 4
    dataset.loc[dataset.extraction_type_class == 'wind-powered', 'extraction_type_class'] = 5
    dataset.loc[dataset.extraction_type_class == 'rope pump', 'extraction_type_class'] = 6
    dataset.loc[dataset.management_group == 'user-group', 'management_group'] = 0
    dataset.loc[dataset.management_group == 'unknown', 'management_group'] = 1
    dataset.loc[dataset.management_group == 'other', 'management_group'] = 1
    dataset.loc[dataset.management_group == 'commercial', 'management_group'] = 2
    dataset.loc[dataset.management_group == 'parastatal', 'management_group'] = 3
    dataset.loc[dataset.payment_type == 'annually', 'payment_type'] = 0
    dataset.loc[dataset.payment_type == 'never pay', 'payment_type'] = 1
    dataset.loc[dataset.payment_type == 'per bucket', 'payment_type'] = 2
    dataset.loc[dataset.payment_type == 'unknown', 'payment_type'] = 3
    dataset.loc[dataset.payment_type == 'other', 'payment_type'] = 3
    dataset.loc[dataset.payment_type == 'on failure', 'payment_type'] = 4
    dataset.loc[dataset.payment_type == 'never pay', 'payment_type'] = 5
    dataset.loc[dataset.payment_type == 'monthly', 'payment_type'] = 6
    dataset.loc[dataset.quality_group == 'good', 'quality_group'] = 0
    dataset.loc[dataset.quality_group == 'salty', 'quality_group'] = 1
    dataset.loc[dataset.quality_group == 'milky', 'quality_group'] = 2
    dataset.loc[dataset.quality_group == 'fluoride', 'quality_group'] = 3
    dataset.loc[dataset.quality_group == 'colored', 'quality_group'] = 4
    dataset.loc[dataset.quality_group == 'unknown', 'quality_group'] = 5
    dataset.loc[dataset.quantity_group == 'enough', 'quantity_group'] = 0
    dataset.loc[dataset.quantity_group == 'insufficient', 'quantity_group'] = 1
    dataset.loc[dataset.quantity_group == 'dry', 'quantity_group'] = 2
    dataset.loc[dataset.quantity_group == 'seasonal', 'quantity_group'] = 3
    dataset.loc[dataset.quantity_group == 'unknown', 'quantity_group'] = 4
    dataset.loc[dataset.source_class == 'groundwater', 'source_class'] = 0
    dataset.loc[dataset.source_class == 'surface', 'source_class'] = 1
    dataset.loc[dataset.source_class == 'unknown', 'source_class'] = 2
    dataset.loc[dataset.waterpoint_type_group == 'communal standpipe', 'waterpoint_type_group'] = 0
    dataset.loc[dataset.waterpoint_type_group == 'hand pump', 'waterpoint_type_group'] = 1
    dataset.loc[dataset.waterpoint_type_group == 'other', 'waterpoint_type_group'] = 2
    dataset.loc[dataset.waterpoint_type_group == 'improved spring', 'waterpoint_type_group'] = 3
    dataset.loc[dataset.waterpoint_type_group == 'cattle trough', 'waterpoint_type_group'] = 4
    dataset.loc[dataset.waterpoint_type_group == 'dam', 'waterpoint_type_group'] = 5
    dataset.loc[dataset.permit.isnull(), 'permit'] = 0
    dataset.loc[dataset.permit == False, 'permit'] = 0
    dataset.loc[dataset.permit == True, 'permit'] = 1
    dataset.loc[dataset.construction_year == 0, 'construction_year'] = np.NaN
    construction_year_avg = dataset['construction_year'].mean() # Calcul de la valeur moyenne
    construction_year_std = dataset['construction_year'].std()  # Calcul de l'écart type
    construction_year_null_count = dataset['construction_year'].isnull().sum() # nombre de valeur nulle
    construction_year_null_random_list = np.random.randint(construction_year_avg - construction_year_std, construction_year_avg + construction_year_std, size=construction_year_null_count)   
    dataset.loc[np.isnan(dataset['construction_year']),'construction_year'] = construction_year_null_random_list    
    dataset['construction_year'] = dataset['construction_year'].astype(int)
    
    dataset.loc[dataset.population == 0, 'population'] = np.NaN
    population_avg = dataset['population'].mean() # Calcul de la valeur moyenne
    population_std = dataset['population'].std()  # Calcul de l'écart type
    population_null_count = dataset['population'].isnull().sum() # nombre de valeur nulle
    population_null_random_list = np.random.randint(population_avg, population_avg + population_std, size=population_null_count)   
    dataset.loc[np.isnan(dataset['population']),'population'] = population_null_random_list    
    dataset['population'] = dataset['population'].astype(int)
    # dataset.loc[dataset.management_group == 'unknown', 'management_group'] = 'other'
    # dataset.loc[dataset.payment_type == 'unknown', 'payment_type'] = 'other'


train['functional']= (train['status_group']=='functional')*1
train['non_functional']= (train['status_group']=='non functional')*1
train['functional_needs_repair']= (train['status_group']=='functional needs repair')*1

train.drop('status_group', axis=1, inplace=True)

BASE_DIR = os.path.abspath('')
# print(BASE_DIR)
train.to_csv(os.path.join(BASE_DIR, 'version_final.csv'))

from sklearn.model_selection import train_test_split
#Séparation des valeurs de train et label (tous les exemples)
X_alltrain = train.values[:, 1:]
y_alltrain = train.values[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X_alltrain, y_alltrain, random_state=42)
print('%i X_train, %i X_test, %i y_train, %i y_test'%(
    X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]))
print('%i X_alltrain, %i y_alltrain'%(X_alltrain.shape[0], y_alltrain.shape[0]))
feature_names=train.columns.tolist()[1:-3]
target_names=["functional","non_functional","functional needs repair"]
print('features:',feature_names)
print('target:',target_names)

# Algorithm à utiliser : XGBClassifier to Random Forest
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, y_train)
for name, score in zip(feature_names, rnd_clf.feature_importances_):
    print('%s: %i%%' %(name, int(score*100)))
    

rnd_clf_short = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)
rnd_clf_short.fit(train[['quality_group','quantity_group','region_code','construction_year']].values, y_alltrain)
